## 코드

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

In [3]:
# plt.rc('font', family='NanumGothicOTF') # For MacOS
plt.rc('font', family='NanumGothic') # For Windows
mpl.rcParams['axes.unicode_minus'] = False
pd.set_option('display.max_columns', None)

In [4]:
raw_path = './csv/raw'
preprocessed_path = './csv/preprocessed'

In [4]:
df1 = pd.read_csv(f'{raw_path}/data_2014.csv',engine = 'python')
df2 = pd.read_csv(f'{raw_path}/data_2015.csv',engine = 'python')
df3 = pd.read_csv(f'{raw_path}/data_2016.csv',engine = 'python') #음주여부 all null
df4 = pd.read_csv(f'{raw_path}/data_2017.csv',engine = 'python')
df5 = pd.read_csv(f'{raw_path}/data_2018.csv')
df6 = pd.read_csv(f'{raw_path}/data_2019.csv', engine = 'python')

In [5]:
def clean_data(df):
    df.columns = df.columns.str.replace(' ', '')
    
    year = df['기준년도'][0]
    print("Numbers of rows in %d before: (%d, %d)" %(year, df.shape[0], df.shape[1]))
    
    df = df.rename(columns={"연령대코드(5세단위)":"연령대코드", 
                        "신장(5Cm단위)":"신장", 
                        "체중(5Kg단위)":"체중",
                        "식전혈당(공복혈당)":"식전혈당",
                        "데이터기준일자":"데이터공개일자",
                        "치석유무":"치석",
                        "(혈청지오티)AST":"AST",
                        "(혈청지오티)ALT":"ALT",})
    
    preserve_columns = ['성별코드', '연령대코드', '신장', '체중', '허리둘레', '수축기혈압',
       '이완기혈압', '식전혈당', '총콜레스테롤', '트리글리세라이드', 'HDL콜레스테롤', 'LDL콜레스테롤',
       '혈색소', '요단백', '혈청크레아티닌', 'AST', 'ALT', '감마지티피', '흡연상태',
       '음주여부']
    
#     drop_column = ['기준년도', '가입자일련번호','시도코드', '시력(좌)', '시력(우)', '청력(좌)', '청력(우)','구강검진수검여부',
#        '치아우식증유무', '결손치유무', '치아마모증유무', '제3대구치(사랑니)이상', '치석', '데이터공개일자']
#     df = df.drop(drop_column, axis = 1, errors='ignore')

    df = df[preserve_columns]
    
    df.drop(df[df['ALT'] >= 500].index, inplace=True)
    df.drop(df[df['AST'] >= 500].index, inplace=True)
    df.drop(df[df['감마지티피'] >= 900].index, inplace=True)
    
    #음주여부
    if  df.loc[df['음주여부'].isnull()].shape[0] < df.shape[0] * 0.5:
        df.loc[df['음주여부'].isnull(), '음주여부'] = 0
    
        if df['음주여부'].dtypes != np.number:
            df.loc[df['음주여부']=='Y', '음주여부'] = 1
            df.loc[df['음주여부']=='N', '음주여부'] = 0
    
    #식전혈당이 없을 시 드랍
    df = df.loc[df['식전혈당'].notnull()]

    #당뇨병: 공복 혈당치 126 mg/dL 이상 당뇨병의 전단계: 공복 혈당치 100 mg/dL 이상
    #저혈당: 혈당치 126 mg/dL 미만
    df.loc[df['식전혈당'] < 100, '식전혈당'] = 0
    df.loc[(df['식전혈당'] >= 100) & (df['식전혈당'] < 126), '식전혈당'] = 1
    df.loc[df['식전혈당'] >= 126, '식전혈당'] = 2
    
    #비정상적인 값 배제
    df = df.loc[df['허리둘레']<140]
    df = df.loc[df['허리둘레']>40]

    #국내의 복부비만 기준은 남자는 90cm 이상, 여자의 경우 85cm 이상
    df_obesity = df[['허리둘레','성별코드']].copy()
    
    df_obesity.loc[(df_obesity['허리둘레']>= 90) & (df_obesity['성별코드']==1), '복부비만'] = 1
    df_obesity.loc[(df_obesity['허리둘레']>= 85) & (df_obesity['성별코드']==2), '복부비만'] = 1
    df_obesity['복부비만'] = df_obesity['복부비만'].fillna(0)

    df['복부비만'] = df_obesity['복부비만']
    
    df_weight = df[['신장', '체중']].copy()

    #체중(kg)을 신장(m)의 제곱으로 나눈 값(체중(kg)/신장(m2))
    temp = df_weight['신장']/100
    df_bmi = round(df_weight['체중'] / (temp * temp), 1)

    #세계보건기구 아시아태평양지역과 대한비만학회
    #BMI < 25 kg/m2 : 저체중+정상 0
    #BMI ≥ 25 kg/m2 : 과체중 1
    #BMI ≥ 30 kg/m2 : 비만 2
    df_bmi.loc[df_bmi < 25] = 0
    df_bmi.loc[(df_bmi >= 25) & (df_bmi < 30)] = 1
    df_bmi.loc[df_bmi >= 30] = 2

    df['비만여부'] = df_bmi

    df = df.drop(['체중', '신장'], axis = 1)
    
    df_pressure = df[['이완기혈압', '수축기혈압']].copy()

    # 정상: 0 / 120~139/80~89 전단계: 1/  140~159/90~99 1기:2 / 160/100 2기:3
    df_pressure.loc[(df_pressure['수축기혈압'] < 120) & (df_pressure['이완기혈압'] < 80), '고혈압 여부'] = 0
    df_pressure.loc[((df_pressure['수축기혈압'] >= 120) & (df_pressure['수축기혈압'] < 140))|
                ((df_pressure['이완기혈압'] >= 80) & (df_pressure['이완기혈압'] < 90)), '고혈압 여부'] = 1
    df_pressure.loc[((df_pressure['수축기혈압'] >= 140) & (df_pressure['수축기혈압'] < 160))|
                ((df_pressure['이완기혈압'] >= 90) & (df_pressure['이완기혈압'] < 99)), '고혈압 여부'] = 2
    df_pressure.loc[(df_pressure['수축기혈압'] >=160) | (df_pressure['이완기혈압'] >= 100), '고혈압 여부'] = 3

    df['고혈압'] = df_pressure['고혈압 여부']
    df = df.drop(['수축기혈압', '이완기혈압'], axis = 1)
    
    if df.loc[df['음주여부'].isnull()].shape[0] > df.shape[0] * 0.5:
        df = df.loc[:, df.columns != '음주여부']
        
    df = df.dropna(axis = 0, how='any')
    
    weight = 0.1
    invalid_cholesterol_idx = []
    for index, row in df.iterrows():
        calculated = row['HDL콜레스테롤'] + row['LDL콜레스테롤'] + (row['트리글리세라이드']/5)
        total = row['총콜레스테롤']
        if (total < (1 - weight) * calculated or (1 + weight) * calculated < total):
            invalid_cholesterol_idx.append(index)
        
    df = df.drop(invalid_cholesterol_idx)
    
    print("Numbers of rows in %d after: (%d, %d)" %(year, df.shape[0], df.shape[1]))
    
    return year, df

In [6]:
df_list = [df1, df2, df3, df4, df5, df6]
df_year = []
df_cleaned = []

for df in df_list:
    year, ddf = clean_data(df)
    df_year.append(year)
    df_cleaned.append(ddf)

Numbers of rows in 2014 before: (1000000, 31)


C:\Users\Public\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.


Numbers of rows in 2014 after: (974332, 19)
Numbers of rows in 2015 before: (1000000, 31)
Numbers of rows in 2015 after: (975085, 19)
Numbers of rows in 2016 before: (1000000, 34)
Numbers of rows in 2016 after: (972275, 18)
Numbers of rows in 2017 before: (1000000, 34)
Numbers of rows in 2017 after: (972186, 19)
Numbers of rows in 2018 before: (1000000, 34)
Numbers of rows in 2018 after: (322063, 19)
Numbers of rows in 2019 before: (1000000, 34)
Numbers of rows in 2019 after: (323548, 19)


In [14]:
df_class_0 = pd.DataFrame(columns = df_cleaned[0].columns)

df_1718 = df_cleaned[3].append(df_cleaned[4])

In [15]:
df_test = df_cleaned[5]
df_test.to_csv(f'{preprocessed_path}/data_test.csv', index=False)

## Without 2016

In [13]:
df_16 = df_1718

for df in df_cleaned[:3]:
    if df.shape[1] != 19:
        continue
    df_12 = df.loc[(df['식전혈당']==1) | (df['식전혈당']==2)]
    df_16 = df_16.append(df_12)
    
df_16 = df_16.sample(frac=1).reset_index(drop=True)

print(df_16['식전혈당'].value_counts())    

df_16.to_csv(f'{preprocessed_path}/data_preprocessed.csv', index=False)

NameError: name 'df_1718' is not defined

In [18]:
X_16 = df_16.loc[:, df_16.columns != '식전혈당']
y_16 = df_16['식전혈당']
y_16=y_16.astype('int')

y_16.value_counts()

1    924629
0    800762
2    234337
Name: 식전혈당, dtype: int64

In [ ]:
oversample = SMOTE()
X_temp, y_temp = oversample.fit_resample(X_16, y_16)

In [ ]:
df_upper = X_temp
df_upper['식전혈당'] = y_temp
df_upper.head()

df_upper.to_csv(f'{file_route}/data_preprocessed_over.csv', index=False)

In [24]:
df_class_0 = df_16[df_16['식전혈당'] == 0]
df_class_1 = df_16[df_16['식전혈당'] == 1]
df_class_2 = df_16[df_16['식전혈당'] == 2]

count_class_2 = df_class_2.shape[0]

df_class_0_under = df_class_0.sample(count_class_2)
df_class_1_under = df_class_1.sample(count_class_2)

df_under = pd.concat([df_class_0_under, df_class_1_under, df_class_2], axis=0)
df_under = df_under.sample(frac=1)

y_under = df_under['식전혈당']

print('UnderSampling 적용 전 레이블 값 분포: \n', pd.Series(y).value_counts())
print('UnderSampling 적용 후 레이블 값 분포: \n', pd.Series(y_under).value_counts())

df_under.to_csv(f'{preprocessed_path}/data_preprocessed_under.csv', index=False)

UnderSampling 적용 전 레이블 값 분포: 
 1    924629
0    800762
2    234337
Name: 식전혈당, dtype: int64
UnderSampling 적용 후 레이블 값 분포: 
 1.0    234337
0.0    234337
2.0    234337
Name: 식전혈당, dtype: int64
